In [1]:
import os

In [2]:
import importlib

In [3]:
import h5py

In [4]:
import pyiron_gpl

In [5]:
from pyiron_base import state, GenericMaster

In [6]:
from pyiron_atomistics import Project

In [7]:
from pyiron_atomistics.lammps.interactive import LammpsInteractive

In [8]:
from pyiron_gpl.elastic.elastic import ElasticMatrixJob

In [9]:
class LammpsInteractiveWithoutOutput(LammpsInteractive):
    def __init__(self, project, job_name):
        super(LammpsInteractiveWithoutOutput, self).__init__(project, job_name)
        self._interactive_disable_log_file = False
    
    def to_hdf(self, hdf=None, group_name=None):
        """

        Args:
            hdf:
            group_name:

        Returns:

        """
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput, self).to_hdf(hdf=hdf, group_name=group_name)
    
    def interactive_flush(self, path="interactive", include_last_step=False):
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput, self).interactive_flush(path=path, include_last_step=include_last_step)

    def interactive_initialize_interface(self):
        if not self._interactive_disable_log_file:
            self._create_working_directory()
        if self.server.run_mode.interactive and self.server.cores == 1:
            lammps = getattr(importlib.import_module("lammps"), "lammps")
            if self._log_file is None:
                self._log_file = os.path.join(self.working_directory, "log.lammps")
            if not self._interactive_disable_log_file:
                self._interactive_library = lammps(
                    cmdargs=["-screen", "none", "-log", self._log_file]
                )
            else:
                self._interactive_library = lammps(
                    cmdargs=["-screen", "none", "-log", "none"]
                )
        else:
            self._interactive_library = LammpsLibrary(
                cores=self.server.cores, working_directory=self.working_directory
            )
        if not all(self.structure.pbc):
            self.input.control["boundary"] = " ".join(
                ["p" if coord else "f" for coord in self.structure.pbc]
            )
        self._reset_interactive_run_command()
        self.interactive_structure_setter(self.structure)
        
    def interactive_close(self):
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput).interactive_close()
            
    def refresh_job_status(self):
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput).refresh_job_status()

In [10]:
class ElasticMatrixJobWithoutFiles(ElasticMatrixJob):
    def __init__(self, project, job_name):
        super(ElasticMatrixJobWithoutFiles, self).__init__(project, job_name)
        self._interactive_disable_log_file = False
        
    def to_hdf(self, hdf=None, group_name=None):
        """

        Args:
            hdf:
            group_name:

        Returns:

        """
        if not self._interactive_disable_log_file:
            super(ElasticMatrixJobWithoutFiles, self).to_hdf(hdf=hdf, group_name=group_name)
    
    def refresh_job_status(self):
        if not self._interactive_disable_log_file:
            super(ElasticMatrixJobWithoutFiles).refresh_job_status()
    
    def collect_output(self):
        if not self._data:
            self.from_hdf()
        self.create_calculator()

        energies = {}
        self._data["id"] = []
        if self.server.run_mode.interactive and not self._interactive_disable_log_file:
            child_id = self.child_ids[0]
            self._data["id"].append(child_id)
            child_job = self.project_hdf5.inspect(child_id)
            energies = {job_name: energy for job_name, energy in zip(self.structure_dict.keys(),
                                                                     child_job["output/generic/energy_tot"])}
        elif self.server.run_mode.interactive and self._interactive_disable_log_file:
            energies = {job_name: energy for job_name, energy in zip(self.structure_dict.keys(),
                                                                     self.ref_job.interactive_cache["energy_tot"])} 
        else:
            for job_id in self.child_ids:
                ham = self.project_hdf5.inspect(job_id)
                en = ham["output/generic/energy_tot"][-1]
                energies[ham.job_name] = en
                self._data["id"].append(ham.job_id)

        self.property_calculator.analyse_structures(energies)
        self._data.update(self.property_calculator._data)
        if not self._interactive_disable_log_file:
            self.to_hdf()
    
    def append(self, job):
        """
        Append a job to the GenericMaster - just like you would append an element to a list.

        Args:
            job (GenericJob): job to append
        """
        if self.status.initialized and not job.status.initialized:
            raise ValueError(
                "GenericMaster requires reference jobs to have status initialized, rather than ",
                job.status.string,
            )
        if job.server.cores >= self.server.cores:
            self.server.cores = job.server.cores
        if job.job_name not in self._job_name_lst:
            self._job_name_lst.append(job.job_name)
            if not self._interactive_disable_log_file:
                self._child_job_update_hdf(parent_job=self, child_job=job)
    
    def pop(self, i=-1):
        """
        Pop a job from the GenericMaster - just like you would pop an element from a list

        Args:
            i (int): position of the job. (Default is last element, -1.)

        Returns:
            GenericJob: job
        """
        job_name_to_return = self._job_name_lst[i]
        job_to_return = self._load_all_child_jobs(
            self._load_job_from_cache(job_name_to_return)
        )
        del self._job_name_lst[i]
        if not self._interactive_disable_log_file:
            with self.project_hdf5.open("input") as hdf5_input:
                hdf5_input["job_list"] = self._job_name_lst
            job_to_return.relocate_hdf5()
        if isinstance(job_to_return, GenericMaster):
            for sub_job in job_to_return._job_object_dict.values():
                self._child_job_update_hdf(parent_job=job_to_return, child_job=sub_job)
        job_to_return.status.initialized = True
        return job_to_return
    
    def _run_if_collect(self):
        """
        Internal helper function the run if collect function is called when the job status is 'collect'. It collects
        the simulation output using the standardized functions collect_output() and collect_logfiles(). Afterwards the
        status is set to 'finished'.
        """


        if not self._interactive_disable_log_file:
            super(ElasticMatrixJobWithoutFiles)._run_if_collect()
        else: 
            self._logger.info(
                "{}, status: {}, finished".format(self.job_info_str, self.status)
            )
            self.collect_output()
            self._logger.info(
                "{}, status: {}, parallel master".format(self.job_info_str, self.status)
            )
            self.update_master()
            # self.send_to_database()

In [11]:
state.settings.configuration

{'user': 'pyiron',
 'resource_paths': ['/Users/janssen/pyiron/resources',
  '/Users/janssen/mambaforge/share/pyiron'],
 'project_paths': [],
 'connection_timeout': 60,
 'sql_connection_string': None,
 'sql_table_name': 'jobs_pyiron',
 'sql_view_connection_string': None,
 'sql_view_table_name': None,
 'sql_view_user': None,
 'sql_view_user_key': None,
 'sql_file': '/Users/janssen/pyiron.db',
 'sql_host': None,
 'sql_type': 'SQLite',
 'sql_user_key': None,
 'sql_database': None,
 'project_check_enabled': False,
 'disable_database': True,
 'credentials_file': None,
 'write_work_dir_warnings': True,
 'config_file_permissions_warning': True}

In [12]:
project = Project("elastic")

In [13]:
project.remove_jobs(recursive=True, silently=True)

In [14]:
potential = '2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1'

In [15]:
structure = project.create.structure.ase.bulk("Fe", cubic=True)
structure

Fe: [0. 0. 0.]
Fe: [1.435 1.435 1.435]
pbc: [ True  True  True]
cell: 
Cell([2.87, 2.87, 2.87])

In [16]:
lmp_mini1 = project.create_job(LammpsInteractiveWithoutOutput, "lmp_mini", delete_existing_job=True)
lmp_mini1.structure = structure
lmp_mini1.potential = potential
lmp_mini1.calc_minimize(pressure=0.0)
lmp_mini1.server.run_mode.interactive = True
lmp_mini1._interactive_disable_log_file = True  # disable lammps.log 
lmp_mini1.run()
lmp_mini1.interactive_close()

The job lmp_mini was saved and received the ID: lmp_mini


/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


In [17]:
lmp_mini1.get_structure()

Fe: [0.00482899 0.00482899 0.00482899]
Fe: [1.435 1.435 1.435]
pbc: [ True  True  True]
cell: 
Cell([[2.8603420110170252, 1.7514543441293517e-16, 1.7514543441293517e-16], [0.0, 2.8603420110170252, 1.7514543441293512e-16], [0.0, 0.0, 2.8603420110170252]])

In [18]:
lmp_elastic = project.create_job(LammpsInteractiveWithoutOutput, "lmp_elastic", delete_existing_job=True)
lmp_elastic.structure = lmp_mini1.get_structure()
lmp_elastic.potential = potential
lmp_elastic.interactive_enforce_structure_reset = True 
lmp_elastic.server.run_mode.interactive = True
lmp_elastic._interactive_disable_log_file = True  # disable lammps.log
elastic = lmp_elastic.create_job(ElasticMatrixJobWithoutFiles, "elastic", delete_existing_job=True)
elastic._interactive_disable_log_file = True  # disable lammps.log
elastic.run()

/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


The job elastic was saved and received the ID: elastic
The job elastic_lmp_elastic was saved and received the ID: elastic_lmp_elastic


In [19]:
def printname(name):
    print(name)

In [20]:
# with h5py.File(project.path + elastic.job_name + ".h5", "r") as f:
#     f.visit(printname)

In [21]:
elastic._data["C"]

array([[184.4207967 , 186.5940429 , 186.5940429 ,   0.        ,
          0.        ,   0.        ],
       [186.5940429 , 184.4207967 , 186.5940429 ,   0.        ,
          0.        ,   0.        ],
       [186.5940429 , 186.5940429 , 184.4207967 ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 165.36013199,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        165.36013199,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        , 165.36013199]])